In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor
#load
train_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')
subsample_size = 500  # subsample subset of data for faster demo, try setting this to much larger values
train_data = train_data.sample(n=subsample_size, random_state=0)
train_data.head()

In [ ]:
label = 'class'
print("Summary of class variable: \n", train_data[label].describe())
save_path = 'agModels-predictClass'  # specifies folder to store trained models
predictor = TabularPredictor(label=label, path=save_path).fit(train_data) #train

In [ ]:
# test
test_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')
y_test = test_data[label]  # values to predict
test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

In [ ]:
#predict
predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file
y_pred = predictor.predict(test_data_nolab)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

In [ ]:
predictor.leaderboard(test_data, silent=True)

# Description of fit():

In [ ]:
pred_probs = predictor.predict_proba(test_data_nolab)
pred_probs.head(5)

In [ ]:
results = predictor.fit_summary(show_plot=True)

In [ ]:
print("AutoGluon infers problem type is: ", predictor.problem_type)
print("AutoGluon identified the following types of features:")
print(predictor.feature_metadata)

In [ ]:
predictor.leaderboard(test_data, silent=True)

# Regression (predicting numeric table columns):

In [ ]:
age_column = 'age'
print("Summary of age variable: \n", train_data[age_column].describe())

In [ ]:
predictor_age = TabularPredictor(label=age_column, path="agModels-predictAge").fit(train_data, time_limit=60)
performance = predictor_age.evaluate(test_data)

In [ ]:
predictor_age.leaderboard(test_data, silent=True)